# SPoOkY Meshes 👻

In [3]:
import base64
import time

import numpy as np
import scipy as sp
import meshplot as mp

import scipy.sparse
import scipy.sparse.linalg
import meshio
import IPython

In [4]:
def load_mesh(filename):
    m = meshio.read(filename)
    v = m.points
    f = m.cells[0].data
    return v, f


def save_mesh(filename, v, f):
    m  = meshio.Mesh(v, f)
    m.write(filename)


# v1, f1 = load_mesh('cw2_meshes/curvatures/plane.obj')
# v2, f2 = load_mesh('cw2_meshes/curvatures/lilium_s.obj')
# v3, f3 = load_mesh('cw2_meshes/decompose/armadillo.obj')
# v4, f4 = load_mesh('cw2_meshes/smoothing/fandisk_ns.obj')
# v5, f5 = load_mesh('cw2_meshes/smoothing/plane_ns.obj')
v7, f7 = load_mesh('sappho-hires.obj')

In [5]:
def normalize(v):
    """
    recenter points
    """
    if not np.any(v):
        return v
    return 2 * (v - v.min()) / (v.max() - v.min()) - 1


def percentile_clip(h, n):
    """
    reject outlier points
    """
    return np.clip(h, np.percentile(h, n), np.percentile(h, 100 - n))

In [6]:
# Unsexy test mesh for debugging
v0 = np.array([
    [ 0, 0, 0],
    [-3, 4, 0],
    [-6, 0, 0],
    [-3,-4, 0],
    [ 3,-4, 0],
    [ 6, 0, 0],
    [ 3, 4, 0],
], dtype=np.float32)

f0 = np.array([
    [0, 1, 2],
    [0, 2, 3],
    [0, 3, 4],
    [0, 4, 5],
    [0, 5, 6],
    [0, 1, 6],
], dtype=np.int32)

# perturbed vertex indices to throw off naive algorithms
# f0 = np.array([
#     [1, 0, 2],
#     [2, 3, 0],
#     [0, 3, 4],
#     [4, 0, 5],
#     [5, 6, 0],
#     [6, 1, 0],
# ], dtype=np.int32)

## Laplace-Beltrami and Friends

### Vertex Area

In [7]:
def triangle_area(v, f):
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    ac = c - a
    bc = c - b

    return np.linalg.norm(np.cross(ac, bc, axis=1), axis=1) / 2


def vertex_area(v, f):
    """
    compute total area about vertices
    3.59ms for 281,724 faces, not bad son
    """
    n = len(v)
    A = np.zeros((3, len(f)))

    area = triangle_area(v, f)

    # set internal angles at vertex location in face array
    # using indexes that have duplicate values to increment doesn't work
    A[0] = area
    A[1] = area
    A[2] = area

    # some esoteric numpy for summing at duplicated indices
    # coo matrices are also an option
    data = A.ravel()
    cols = f.T.ravel()

    M = np.zeros(n)
    np.add.at(M, cols, data)

    return sp.sparse.diags(M)


def barycentric_mass(v, f):
    return vertex_area(v, f) / 3


# %timeit vertex_area(v4, f4)


M = barycentric_mass(v0, f0)
assert list(M.diagonal()) == [24, 8, 8, 8, 8, 8, 8]

### Cotangent operator

In [8]:
def cotangent_curvature(v, f):
    """
    we can sum the contribution of all adjacent angles,
    then subtract the contribution of all interior angles.
    
    using edges would probably avoid some redundant work here (as above).

    133ms seconds for 281,724 faces. igl is 52.2ms.
    """
    n = len(v)
    
    # indices
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    # vectors
    ab = b - a
    ac = c - a
    bc = c - b
    
    # big chungus cotangent computation
    abc = np.einsum('ij,ij->i', ab, ac) / np.linalg.norm(np.cross( ab, ac, axis=1), axis=1)
    bac = np.einsum('ij,ij->i',-ab, bc) / np.linalg.norm(np.cross(-ab, bc, axis=1), axis=1)
    cab = np.einsum('ij,ij->i',-ac,-bc) / np.linalg.norm(np.cross(-ac,-bc, axis=1), axis=1)

    # set weights for opposite edges
    # a csr_matrix will sum the quantities for us!
    data = np.hstack([cab, abc, bac])
    rows = np.hstack([i,   j,   k, ])
    cols = np.hstack([j,   k,   i, ])
    T = sp.sparse.csr_matrix((data, (rows, cols)))

    # mad gains by flipping across the diagonal ;)
    T = T + T.T

    # Nearly there. sum the rows and use as diagonal.
    S = sp.sparse.diags(T.sum(axis=1).A.ravel())
    T = T - S

    # i'm never doing this again
    # divide by two as we have computed k_1 + k_2
    return T / 2


# %timeit cotangent_curvature(v7, f7)

C = cotangent_curvature(v0, f0)
T = np.diag(np.round((180 / np.pi) * C.todense()))

assert list(T) == [-205, -73, -60, -73, -73, -60, -73]
assert np.sum(C) < 1e-6

### Check that mean curvature works

In [157]:
def mean_curvature_metric(v, f):
    M = barycentric_mass(v, f)
    C = cotangent_curvature(v, f)
    Mi = sp.sparse.diags(1 / M.diagonal())
    Hn = -Mi @ C @ v
    H  = np.linalg.norm(Hn, axis=1)
    return H


c7 = percentile_clip(mean_curvature_metric(v7, f7), 5)

mp.plot(v7, f7, c7)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(28.620000…

## TV Divergence & Gradient Operators

hack this together

$$
\mathbf{G} f\left(\mathbf{t}_{i j k}\right)=\left(\begin{array}{cc}
\mathbf{v}_{j}^{\top}-\mathbf{v}_{i}^{\top} \\
\mathbf{v}_{k}^{\top}-\mathbf{v}_{i}^{\top}
\end{array}\right)^{\top}\left(\begin{array}{cc}
\left\|e_{i j}\right\|^{2} & \left\langle e_{i j}, e_{i k}\right\rangle \\
\left\langle e_{i j}, e_{i k}\right\rangle & \left\|e_{i k}\right\|^{2}
\end{array}\right)\left(\begin{array}{l}
f\left(\mathbf{v}_{j}\right)-f\left(\mathbf{v}_{i}\right) \\
f\left(\mathbf{v}_{k}\right)-f\left(\mathbf{v}_{i}\right)
\end{array}\right)
$$

In [158]:
def twonormest(A):
    """
    square root of largest singular value of A?!?
    https://en.wikipedia.org/wiki/Matrix_norm
    """
    [e] = scipy.sparse.linalg.svds(A, k=1, return_singular_vectors=False)
    return np.sqrt(e)


def tv_gradient_operator(v, f, axis):
    # indices
    i, j, k = f.T

    # edges?
    e_ij = v[j] - v[i]
    e_ik = v[k] - v[i]
    
    # hstack and reshape into F x 2 x 3
    # TODO: can we remove this swapaxes?
    A = np.hstack([
        e_ij,
        e_ik,
    ]).reshape(len(f), 2, 3).swapaxes(1, 2)
    
    e_ij2 = np.sum(e_ij ** 2, axis=1)
    e_ik2 = np.sum(e_ik ** 2, axis=1)
    
    e_min = np.min(np.hstack([np.sqrt(e_ij2), np.sqrt(e_ik2)]))

    e_ijik = np.einsum('ij,ij->i', e_ij, e_ik)

    B = np.hstack([
        [e_ij2, e_ijik, e_ijik, e_ik2],             
    ]).reshape(2, 2, len(f)).T

    # our 'function' is the X coordinate for now
    C = np.array([
        v[j][:, axis] - v[i][:, axis],
        v[k][:, axis] - v[i][:, axis],
    ]).reshape(len(f), 2, 1)

    D = A @ B @ C
    
    rows = np.repeat(np.arange(0, len(f)), 3)
    cols = f.ravel()
    data = D.ravel()

    G = sp.sparse.csr_matrix((data, (rows, cols)))
    
    # compute tau, sigma while we're in toon
    N = twonormest(G)
    t = e_min / N
    
    return G, t


def tv_gradient_operators(v, f, axis=0):
    """
    return G and adjoint D
    """
    G, t = tv_gradient_operator(v, f, axis)
    A = barycentric_mass(v, f)
    T = sp.sparse.diags(triangle_area(v, f))
    Ai = sp.sparse.diags(1 / A.diagonal())
    D = -Ai @ G.T @ T
    return G, D, t


# Test that our unsexy mesh produces the correct value for the first row
G, t = tv_gradient_operator(v0, f0)
assert np.allclose(G.todense().A[0], [2169, -732, 0, 0, 0, 0, 0])

# test that we get a value for tau / sigma
assert np.allclose(t, 0.083425544)

In [273]:
def prox_f(q):
    """
    project q onto the l2 ball
    """
    qT = q.T
    qT_norm = np.linalg.norm(qT, axis=0)
    indices = qT_norm > 1
    qT[:, indices] = qT[:, indices] / qT_norm[indices]
    return qT.T

A = np.array([
    [0  ,   1,   2],  # normalized
    [1/2, 1/2, 1/2],  # unchanged
    [1/3, 1/3, 1/3],  # unchanged
    [1/4, 1/4, 1/4],  # unchanged
])
proj = prox_f(A)

assert np.allclose(proj[0], [0. , 0.4472136 , 0.89442719])
assert np.allclose(proj[1], A[1])
assert np.allclose(proj[2], A[2])
assert np.allclose(proj[3], A[3])

In [267]:

def solve(v, f, axis):
    """
    The algorithm simply evolves the input signal by N discrete steps along the TV flow;
    each iteration moves a step forward, with diffusion time equal to α.
    As changes happen quickly for small t and tend to become slower for larger t
    we iteratively increase the step size α of the evolution.
    
    Subsequently, the spectral representation φt is constructed incrementally using finite differences,
    such that the integral of Eq. (12) becomes a simple weighted sum over the φt.
    We give selection strategies for α and N in Appendix B.4.
    """
    t = 5  # number of time steps ~= number of features?!
    
    u_0 = v[:, axis]  # input signal
    u_t = np.zeros((t, *u_0.shape))  # feature stack
    p_t = np.zeros_like(u_t)  # spectral representation

    alpha = 0.1  # step size - not sure what this should be

    # operators - these may change on each iteration and need to be recomputed...
    G, D, tau = tv_gradient_operators(v, f, axis)
    sigma = tau
    
    # just 0.5. phew!
    theta = 0.5
    
    u = u_0
    
    def prox_g(u):
        output = u + (tau / alpha) + u_0
        output = output / (1 + (tau / alpha))
        return output

    for i in range(0, t):
        
        # initialize q
        q = np.zeros_like(f[:, axis])

        # algorithm 2
        while True:
            u_next = prox_g(u - tau * D @ q)
            u_bar  = u_next + theta * (u_next - u)
            q_next = prox_f(q + sigma * G @ u_bar)

            if np.linalg.norm(u_next - u) < 1:
                print("converged!")
                u_t[i] = u_next
                break  # done?!

            u = u_next
            q = q_next
        
        p_t[i] = u_t[i] - u_t[i-1]
        u_0    = u_t[i]
    return p_t


p_t = solve(v7, f7, axis=0)

v7_t = np.vstack([p_t[4], v7[:, 1], v7[:, 2]]).T

mp.plot(v7_t, f7)

converged!
converged!
converged!
converged!
converged!


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12947783.…